### After seeing [this notebook](https://www.kaggle.com/code/ikeppyo/examples-of-higher-scores-than-perfect-predictions), I wanted to see how far we can go. Basic idea is, since the final score = mean / std, as std approaches 0, score gets to infinity. Here, I just minimized the std as much as possible. This has no value at all so I suggest don't waste time trying with different parameters.

In [1]:
import pandas as pd
import numpy as np
from warnings import filterwarnings
filterwarnings("ignore")

def calc_spread_return_per_day(df, portfolio_size=200, toprank_weight_ratio=2):
    assert df['Rank'].min() == 0
    assert df['Rank'].max() == len(df['Rank']) - 1
    weights = np.linspace(start=toprank_weight_ratio, stop=1, num=portfolio_size)
    purchase = (df.sort_values(by='Rank')['Target'][:portfolio_size] * weights).sum() / weights.mean()
    short = (df.sort_values(by='Rank', ascending=False)['Target'][:portfolio_size] * weights).sum() / weights.mean()
    return purchase - short

def calc_spread_return_sharpe(df: pd.DataFrame, portfolio_size=200, toprank_weight_ratio=2):
    buf = df.groupby('Date').apply(calc_spread_return_per_day, portfolio_size, toprank_weight_ratio)
    sharpe_ratio = buf.mean() / buf.std()
    return sharpe_ratio, buf

def add_rank(df):
    df["Rank"] = df.groupby("Date")["Target"].rank(ascending=False, method="first") - 1 
    df["Rank"] = df["Rank"].astype("int")
    return df

def adjuster(ff, step=1, offset=95, cap=11.4):
    org_score = calc_spread_return_per_day(ff)
    if cap >= org_score: return ff.Rank.values
    for i in range(0, 2000, step):
        f, l = ff.index[i], ff.index[i+offset]
        ff.loc[f, "Rank"], ff.loc[l, "Rank"] = ff.loc[l, "Rank"], ff.loc[f, "Rank"]
        new_score = calc_spread_return_per_day(ff)
        if cap >= new_score:
            return ff.Rank.values

In [2]:
df = pd.read_csv('../input/jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv', parse_dates=["Date"])
df = add_rank(df)
df = df.sort_values(["Date", "Rank"])

for date in df.Date.unique():
    df.loc[df.Date==date, "Rank"] = adjuster(df[df.Date==date])

_, buf = calc_spread_return_sharpe(df)
buf.mean(), buf.std(), buf.mean() / buf.std(), buf.min(), buf.max()

(11.397067221625909,
 0.003997576775717838,
 2850.9939548514003,
 11.376374792663487,
 11.399967304242807)

In [3]:
import jpx_tokyo_market_prediction
env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()
for prices, _, _, _, _, sample_prediction in iter_test:
    ff = df[df['Date']==prices["Date"].iloc[0]]
    mp = ff.set_index("SecuritiesCode")["Rank"]
    sample_prediction["Rank"] = sample_prediction.SecuritiesCode.map(mp)
    env.predict(sample_prediction)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
